# Подготовка данных

In [1]:
!pip install implicit
!pip install lightfm

     |████████████████████████████████| 1.1MB 5.6MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp36-cp36m-linux_x86_64.whl size=3419443 sha256=652f8ade12d3feeafa647e300b8b80a82a63fceb375ab3ec7b392a8161e2b5c4
  Stored in directory: /root/.cache/pip/wheels/bf/d4/ec/fd4f622fcbefb7521f149905295b2c26adecb23af38aa28217
Successfully built implicit
     |████████████████████████████████| 317kB 5.1MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp36-cp36m-linux_x86_64.whl size=704293 sha256=83ab3eb31351cd32e924b93d6d36072fb202ed5b11e9b3925e6bd0580a9f0580
  Stored in directory: /root/.cache/pip/wheels/c6/64/d4/673c7277f71ac4c5ad4835b94708c01b653ef2d3aa78ef20aa
Successfully built lightfm


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!cp -R  /content/drive/MyDrive/Recomendation/srcrec  /

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
src_path = os.path.join(module_path, "srcrec")
if module_path not in sys.path:
    sys.path.append(module_path)
    sys.path.append(src_path)
    
from srcrec.metrics import precision_at_k, recall_at_k
from srcrec.utils import prefilter_items
from srcrec.recommenders import MainRecommender

In [5]:
base_path = "/content/drive/MyDrive/Recomendation/"

data = pd.read_csv(base_path+'retail_train.csv')
item_features = pd.read_csv(base_path+'product.csv')
user_features = pd.read_csv(base_path+'hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# retail_train


In [6]:
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2396804 entries, 0 to 2396803
Data columns (total 12 columns):
 #   Column             Dtype  
---  ------             -----  
 0   user_id            int64  
 1   basket_id          int64  
 2   day                int64  
 3   item_id            int64  
 4   quantity           int64  
 5   sales_value        float64
 6   store_id           int64  
 7   retail_disc        float64
 8   trans_time         int64  
 9   week_no            int64  
 10  coupon_disc        float64
 11  coupon_match_disc  float64
dtypes: float64(4), int64(8)
memory usage: 219.4 MB


In [8]:
data.describe()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
count,2.396804e+06,2.396804e+06,2.396804e+06,2.396804e+06,2.396804e+06,2.396804e+06,2.396804e+06,2.396804e+06,2.396804e+06,2.396804e+06,2.396804e+06,2.396804e+06
mean,1.271904e+03,3.336643e+10,3.639770e+02,2.827247e+06,1.003763e+02,3.100821e+00,3.048227e+03,-5.400708e-01,1.561714e+03,5.268156e+01,-1.638696e-02,-2.897905e-03
std,7.265644e+02,4.284798e+09,1.759385e+02,3.732798e+06,1.152379e+03,4.210229e+00,8.785542e+03,1.245824e+00,4.015691e+02,2.513310e+01,2.168615e-01,3.974618e-02
min,1.000000e+00,2.698485e+10,1.000000e+00,2.567100e+04,0.000000e+00,0.000000e+00,1.000000e+00,-1.300200e+02,0.000000e+00,1.000000e+00,-5.593000e+01,-7.700000e+00
25%,6.550000e+02,3.008714e+10,2.160000e+02,9.169930e+05,1.000000e+00,1.290000e+00,3.300000e+02,-6.900000e-01,1.307000e+03,3.200000e+01,0.000000e+00,0.000000e+00
50%,1.271000e+03,3.241998e+10,3.660000e+02,1.027569e+06,1.000000e+00,2.000000e+00,3.700000e+02,-2.000000e-02,1.614000e+03,5.300000e+01,0.000000e+00,0.000000e+00
75%,1.914000e+03,3.514580e+10,5.150000e+02,1.132178e+06,1.000000e+00,3.490000e+00,4.220000e+02,0.000000e+00,1.844000e+03,7.400000e+01,0.000000e+00,0.000000e+00
max,2.500000e+03,4.165679e+10,6.630000e+02,1.802456e+07,8.963800e+04,8.400000e+02,3.428000e+04,3.990000e+00,2.359000e+03,9.500000e+01,0.000000e+00,0.000000e+00


## product

In [9]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [10]:
item_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92353 entries, 0 to 92352
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   item_id               92353 non-null  int64 
 1   manufacturer          92353 non-null  int64 
 2   department            92353 non-null  object
 3   brand                 92353 non-null  object
 4   commodity_desc        92353 non-null  object
 5   sub_commodity_desc    92353 non-null  object
 6   curr_size_of_product  92353 non-null  object
dtypes: int64(2), object(5)
memory usage: 4.9+ MB


в столбце "urr_size_of_product" есть пустые строки

In [11]:
item_features.describe()

,item_id,manufacturer
count,9.235300e+04,92353.00000
mean,5.328353e+06,1739.22833
std,5.359937e+06,1818.26957
min,2.567100e+04,1.00000
25%,9.706280e+05,328.00000
50%,1.621091e+06,1094.00000
75%,9.704770e+06,2264.00000
max,1.831630e+07,6477.00000


In [12]:
item_features['sub_commodity_desc'].unique()

array(['ICE - CRUSHED/CUBED', 'NO SUBCOMMODITY DESCRIPTION',
       'BREAD:ITALIAN/FRENCH', ..., 'SEMI MOIST CAT FOOD: BOX',
       'BERRIES OTHER', 'ROSES OTHER'], dtype=object)

In [13]:
item_features['sub_commodity_desc'].nunique()

2383

In [14]:
item_features['curr_size_of_product'].unique()

array(['22 LB', ' ', '50 OZ', ..., '6.3 IN', '35 LD', '2 LTR PET'],
      dtype=object)

In [15]:
item_features['curr_size_of_product'].nunique()

4345

## hh_demographic

In [16]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [17]:
user_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 801 entries, 0 to 800
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   age_desc             801 non-null    object
 1   marital_status_code  801 non-null    object
 2   income_desc          801 non-null    object
 3   homeowner_desc       801 non-null    object
 4   hh_comp_desc         801 non-null    object
 5   household_size_desc  801 non-null    object
 6   kid_category_desc    801 non-null    object
 7   user_id              801 non-null    int64 
dtypes: int64(1), object(7)
memory usage: 50.2+ KB


In [18]:
data.user_id.nunique()

2499

In [19]:
u_all = set(data.user_id.unique().tolist())
u_inf = set(user_features.user_id.unique().tolist())

len(u_all.intersection(u_inf))

801

<!-- Пропусков нет, но данные охватывают только 32.05% всех пользователей -->

# Фильтрация

### Убираем 3 самых популярных

In [20]:
popularity = data.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [21]:
item_features.loc[item_features['item_id'].isin(top_5000[:3])]

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
57181,6533889,69,MISC SALES TRAN,Private,COUPON/MISC ITEMS,GASOLINE-REG UNLEADED,
57216,6534166,69,MISC SALES TRAN,Private,COUPON/MISC ITEMS,GASOLINE-REG UNLEADED,
57221,6534178,69,KIOSK-GAS,Private,COUPON/MISC ITEMS,GASOLINE-REG UNLEADED,


In [22]:
data.loc[data['item_id'].isin(top_5000[:3]), 'item_id'] = 999999

### Уберем которые не продавались за последние 12 месяцев

In [23]:
curr_week = data['week_no'].values.max()
data.loc[data['week_no']<curr_week-53, 'item_id'] = 999999

### Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоят < 1$

In [24]:
mean_price = data.groupby('item_id')['quantity', "sales_value"].sum()
mean_price["mean_price"] = mean_price["sales_value"] / mean_price["quantity"]
cheap_list = mean_price.loc[mean_price["mean_price"] < 0.99].index.tolist()
data.loc[data['item_id'].isin(cheap_list), 'item_id'] = 999999

### Cамые непопулярные

In [25]:
popularity_pu = data.groupby('item_id')['user_id'].nunique().reset_index()
popularity_pu.rename(columns={'user_id': 'share_unique_users'}, inplace=True)
popularity_pu['share_unique_users'] = popularity_pu['share_unique_users'] / data['user_id'].nunique()
popularity_pu.sort_values('share_unique_users', ascending=False, inplace=True)
popularity_pu.head(3)

,item_id,share_unique_users
18291,999999,1.000000
16754,981760,0.529012
17855,995242,0.469388


In [26]:
popularity_pu.tail(5)

,item_id,share_unique_users
33700,2039496,0.0004
33699,2039122,0.0004
33698,2038438,0.0004
33697,2038349,0.0004
31634,1395707,0.0004


In [27]:
popularity_pu.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63269 entries, 18291 to 31634
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   item_id             63269 non-null  int64  
 1   share_unique_users  63269 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 1.4 MB


In [28]:
popularity_pu[popularity_pu['share_unique_users'] < 0.005]

,item_id,share_unique_users
13270,939383,0.004802
12193,926959,0.004802
47061,9884579,0.004802
7939,877525,0.004802
24787,1076889,0.004802
...,...,...
33700,2039496,0.000400
33699,2039122,0.000400
33698,2038438,0.000400
33697,2038349,0.000400


In [29]:
unpop_list = popularity_pu[popularity_pu['share_unique_users'] < 0.005].item_id.tolist()
data.loc[data['item_id'].isin(unpop_list), 'item_id'] = 999999

## Feature Engineering

### Средний чек

In [30]:
bills = data.groupby(["basket_id", 'user_id'])["sales_value"].sum()
bills = bills.reset_index(level=[0,1])
bills.head()

,basket_id,user_id,sales_value
0,26984851472,2375,5.91
1,26984851516,2375,12.43
2,26984896261,1364,11.37
3,26984905972,1130,2.55
4,26984945254,1173,3.53


In [31]:
bills_avg = data.groupby(['user_id'])["basket_id"].count()
bills_avg = bills_avg.reset_index()
bills_avg.columns=['user_id', 'bills_cnt']
bills_avg

,user_id,bills_cnt
0,1,1589
1,2,655
2,3,889
3,4,301
4,5,219
...,...,...
2494,2496,1420
2495,2497,1938
2496,2498,833
2497,2499,969


In [32]:
bills_sum = bills.groupby(['user_id'])["sales_value"].sum()

In [33]:
bills_avg['bill_sum'] = bills_sum.values
bills_avg['bill_avg'] = bills_avg['bill_sum'] / bills_avg['bills_cnt']
bills_avg.head()

,user_id,bills_cnt,bill_sum,bill_avg
0,1,1589,3959.91,2.492077
1,2,655,1823.45,2.783893
2,3,889,2594.30,2.918223
3,4,301,1200.11,3.987076
4,5,219,749.09,3.420502


In [34]:
user_features = user_features.merge(bills_avg[["user_id", "bill_avg"]], on='user_id', how='left')
user_features.head()

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id,bill_avg
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1,2.492077
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7,2.673405
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8,2.798250
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13,5.693402
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16,2.927602


Последняя покупка

In [35]:
bills_per_week = data.groupby(['user_id'])["week_no"].max()

bills_per_week = bills_per_week.reset_index()
bills_per_week.columns = ["user_id", "week_last_bill"]
bills_per_week.head(2)

,user_id,week_last_bill
0,1,95
1,2,90


In [36]:
user_features = user_features.merge(bills_per_week, on='user_id', how='left')
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id,bill_avg,week_last_bill
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1,2.492077,95
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7,2.673405,95


### Количество покупок в неделю

In [37]:
bills_week = data.groupby(['user_id', 'week_no'])["item_id"].count()
bills_week

user_id  week_no
1        8          30
         10         12
         13         12
         14         23
         15         17
                    ..
2500     91         11
         92         25
         93         26
         94         19
         95         36
Name: item_id, Length: 114839, dtype: int64

In [38]:
bills_week = bills_week.reset_index()
bills_week.columns = ["user_id", "week_no", "item_cnt"]
bills_week.head(3)

,user_id,week_no,item_cnt
0,1,8,30
1,1,10,12
2,1,13,12


In [39]:
bills_count = bills_week.groupby("user_id")["item_cnt"].sum()


In [40]:
bills_per_week['item_cnt'] = bills_count.values
old_bill = data.groupby(['user_id'])["week_no"].min()
bills_per_week["week_oldest_bill"] = old_bill.values

In [41]:
bills_per_week["user_int"] = 95 - bills_per_week["week_oldest_bill"]
bills_per_week["items_per_week"] = bills_per_week["item_cnt"] / bills_per_week["user_int"]
bills_per_week.head(5)

,user_id,week_last_bill,item_cnt,week_oldest_bill,user_int,items_per_week
0,1,95,1589,8,87,18.264368
1,2,90,655,15,80,8.187500
2,3,92,889,17,78,11.397436
3,4,90,301,16,79,3.810127
4,5,85,219,13,82,2.670732


In [42]:
user_features = user_features.merge(bills_per_week[["user_id", "items_per_week"]], on='user_id', how='left')
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id,bill_avg,week_last_bill,items_per_week
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1,2.492077,95,18.264368
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7,2.673405,95,11.780220


## Сохраняем данные

In [ ]:
data.to_csv(base_path+"df_data.csv", index=False)
user_features.to_csv(base_path+"user_features.csv", index=False)
item_features.to_csv(base_path+"item_features.csv", index=False)